In [1]:
!pip install pycocotools

  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=272453 sha256=93ef9690f0d9db3a8f706595b464cccba8174cc6013d0d554a8c30f1c6fb21a5
  Stored in directory: /root/.cache/pip/wheels/bc/cf/1b/e95c99c5f9d1648be3f500ca55e7ce55f24818b0f48336adaf
Successfully built pycocotools


In [2]:
import os
#os.getcwd()
import pycocotools
os.chdir("../input/help-files")
import numpy as np
import torch
import torch.utils.data
from PIL import Image
import pandas as pd
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from engine import train_one_epoch, evaluate
import utils
import transforms as T
print(os.getcwd())

/kaggle/input/help-files


In [3]:
#import train.csv and remove 'no findings' samples
data_df = pd.read_csv('/kaggle/input/vinbigdata-original-image-dataset/vinbigdata/train.csv')
data_df.head()
#len(data_df) #67914
# Remove no finding samples
data_df = data_df.loc[data_df['class_id'] != 14].reset_index(drop=True)
data_df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height
0,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0,2080,2336
1,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0,2304,2880
2,1c32170b4af4ce1a3030eb8167753b06,Pleural thickening,11,R9,627.0,357.0,947.0,433.0,2540,3072
3,0c7a38f293d5f5e4846aa4ca6db4daf1,ILD,5,R17,1347.0,245.0,2188.0,2169.0,2285,2555
4,47ed17dcb2cbeec15182ed335a8b5a9e,Nodule/Mass,8,R9,557.0,2352.0,675.0,2484.0,2568,3353


In [4]:
#adjust bounding boxes according to the resized images dimension ( 512 X 512)
data_df['x_min'] = data_df.apply(lambda row: (row.x_min)/row.height*512, axis =1)
data_df['y_min'] = data_df.apply(lambda row: (row.y_min)/row.width*512, axis =1)

data_df['x_max'] = data_df.apply(lambda row: (row.x_max)/row.height*512, axis =1)
data_df['y_max'] = data_df.apply(lambda row: (row.y_max)/row.width*512, axis =1)

In [5]:
#check if bounding box valuse are not consistent and remove those records in case of inconsistency
def calculate_iou(bbox1, bbox2):
    # Coordinates must be consistent
    #print(bbox1['x_min'])
    assert(bbox1['x_min'] < bbox1['x_max'])
    assert(bbox1['y_min'] < bbox1['y_max'])
    assert(bbox2['x_min'] < bbox2['x_max'])
    assert(bbox2['y_min'] < bbox2['y_max'])
    
    # Calculate coordinates of the top left corner of the intersection area
    x_top_left = max(bbox1['x_min'], bbox2['x_min'])
    y_top_left = max(bbox1['y_min'], bbox2['y_min'])
    
    # Calculate coordinates of the bottom right corner of the intersection area
    x_bottom_right = min(bbox1['x_max'], bbox2['x_max'])
    y_bottom_right = min(bbox1['y_max'], bbox2['y_max'])
    
    # Calculate IOU
    area_bbox1 = ((bbox1['x_max'] - bbox1['x_min']) * (bbox1['y_max'] - bbox1['y_min']))
    assert area_bbox1 > 0
    area_bbox2 = ((bbox2['x_max'] - bbox2['x_min']) * (bbox2['y_max'] - bbox2['y_min']))
    assert area_bbox2 > 0
    
    if x_top_left > x_bottom_right or y_top_left > y_bottom_right:
        return 0.0, area_bbox1, area_bbox2
    
    area_intersection = (x_bottom_right - x_top_left) * (y_bottom_right - y_top_left) 
    assert area_intersection >= 0
    
    area_union = area_bbox1 + area_bbox2 - area_intersection
    
    iou = area_intersection / area_union
    assert iou >= 0.0 and iou <= 1.0
    
    return iou, area_bbox1, area_bbox2

# Remove bounding boxes with high IOU and same class
def remove_bboxs(df, threshold=0.5):
    img_ids = df['image_id'].unique()
    new_records = list()

    for img_id in img_ids:
        records = df[df['image_id'] == img_id].reset_index(drop=True)
        to_drop = list()
        size = records.shape[0]
        for i in range(size-1):
            if i in to_drop:
                continue
            bbox1 = records.iloc[[i],:]
            bbox1 = bbox1.to_dict('records')[0]
            for j in range(i+1, size):
                bbox2 = records.iloc[[j],:]
                bbox2 = bbox2.to_dict('records')[0]

                iou, bb1_area, bb2_area = calculate_iou(bbox1, bbox2)
                if iou >= threshold and bbox1['class_id'] == bbox2['class_id']:
                    if bb1_area >= bb2_area:
                        to_drop.append(i)
                        break
                    else:
                        to_drop.append(j)
        records = records.loc[~records.index.isin(to_drop)]
        new_records.append(records)

    return pd.concat(new_records)

In [6]:
print(data_df.shape)
data_df = remove_bboxs(data_df, threshold = 0.1)
print(data_df.shape)

(36096, 10)
(20747, 10)


In [7]:
testt = data_df
testt.head()
testt = testt.drop_duplicates('image_id')
testt.shape
# testt.head()
# len(testt)
# img_in_testt = testt['image_id'] + '.jpg'
# img_in_testt = img_in_testt.to_list()
# len(img_in_testt)

(4394, 10)

In [8]:
len(testt)

4394

In [9]:
#train test split
from sklearn.model_selection import train_test_split

train, valid = train_test_split(testt, test_size=0.2)
len(train), len(valid)
valid

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height
2,127f66766091998a72268caf4ec34bef,Cardiomegaly,3,R8,196.937973,294.087343,433.960666,370.262028,2702,2644
0,47110277377a779131d0e08d4389a503,Pulmonary fibrosis,13,R8,108.361431,115.735109,135.218921,136.950055,2703,3298
2,6747991aa8970e35ec44fe90b309d627,Pleural thickening,11,R10,107.666667,81.637795,128.000000,96.957480,2540,3072
1,18533a53425600e4da1e36085cbad104,Aortic enlargement,0,R10,259.056180,137.117318,301.662921,186.458101,2864,2848
1,6312578be73812b1634727a012980bc6,Pleural thickening,11,R8,126.752089,87.094801,206.618384,107.840979,2616,2872
...,...,...,...,...,...,...,...,...,...,...
2,908cff12e3ce717c4fc6cba8290b89a6,Nodule/Mass,8,R9,74.208589,347.491409,87.754601,365.305842,2328,2608
1,c1ca66539955f94f6dbd74fb5ba7208d,ILD,5,R10,118.833333,185.166667,191.666667,296.833333,3072,3072
0,bf2d10fe88254cf97b08fab2e7c80232,Cardiomegaly,3,R8,184.015199,307.565714,354.087397,409.234286,2800,3158
1,bd3f8d06c2eab0bfe5428dfe3f9a1499,Pleural effusion,10,R9,18.178404,98.934019,146.779343,460.668380,2334,3408


In [10]:
img_in_train = train['image_id'] + '.jpg'
img_in_train = img_in_train.to_list()

img_in_valid = valid['image_id'] + '.jpg'
img_in_valid = img_in_valid.to_list()
len(img_in_train), len(img_in_valid)

(3515, 879)

In [11]:
class VinBigDataset(torch.utils.data.Dataset):
    def __init__(self, root, li, data_file, transforms=None):
      self.root = root
      self.li=li   
      self.transforms = transforms 
      #self.imgs = sorted(os.listdir(os.path.join(root, "train")))
      self.imgs = li  
      self.path_to_data_file = data_file
      #img_path = os.path.join(self.root, "train", self.imgs[0])
      
      
      
    def __getitem__(self, idx):
      # load images and bounding boxes
      img_path = os.path.join(self.root, "train", self.imgs[idx])
      img = Image.open(img_path).convert("RGB")
      img = img.resize((512,512))
      box_list = parse_one_annot(self.path_to_data_file, 
      self.imgs[idx])
      
     
      boxes = torch.as_tensor(box_list, dtype=torch.float32)
     
      num_objs = len(box_list)
      # there is only one class
      #labels = torch.ones((num_objs,), dtype=torch.int64)
      labels= parse_label(self.path_to_data_file, self.imgs[idx])  
      
      #labels= torch.from_numpy(data["class_id"].values)

      image_id = torch.tensor([idx])
      area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:,
      0])
      # suppose all instances are not crowd
      iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
      target = {}
      target["boxes"] = boxes
      target["labels"] = labels
      target["image_id"] = image_id
      target["area"] = area
      target["iscrowd"] = iscrowd
      if self.transforms is not None:
        img, target = self.transforms(img, target)
      return img, target
    def __len__(self):
         return len(self.imgs)



def get_transform(train):
   transforms = []
   # converts the image, a PIL image, into a PyTorch Tensor
   transforms.append(T.ToTensor())
   if train:
      #pass
      # during training, randomly flip the training images
      # and ground-truth for data augmentation
      transforms.append(T.RandomHorizontalFlip(0.5))
   return T.Compose(transforms)

def parse_one_annot(path_to_data_file, filename):
   #data = pd.read_csv(path_to_data_file)
   data = path_to_data_file
   ab= data["image_id"] + '.jpg'
   boxes_array = data[ab == filename][["x_min", "y_min", "x_max", "y_max"]].values
   
   return boxes_array

def parse_label(path_to_data_file, filename):
   #print(path_to_data_file)
   #data = pd.read_csv(path_to_data_file)
   data = path_to_data_file
   ab= data["image_id"] + '.jpg'
   label = []
   label = data[ab == filename]["class_id"].values
   label = torch.from_numpy(label)
   return label


# use our dataset and defined transformations

dataset_train = VinBigDataset(root= "/kaggle/input/vinbigdata-original-image-dataset/vinbigdata", 
                           li = img_in_train, data_file= train,transforms = get_transform(train=True))
dataset_valid = VinBigDataset(root= "/kaggle/input/vinbigdata-original-image-dataset/vinbigdata", 
                           li = img_in_valid, data_file= valid,transforms = get_transform(train=False))
len(dataset_train), len(dataset_valid)


(3515, 879)

In [12]:

data_loader_train = torch.utils.data.DataLoader(
              dataset_train, batch_size=10, shuffle=True, num_workers=4,
              collate_fn=utils.collate_fn)
data_loader_valid = torch.utils.data.DataLoader(
              dataset_valid, batch_size=10, shuffle=True, num_workers=4,
              collate_fn=utils.collate_fn)

len(data_loader_train), len(data_loader_valid)

(352, 88)

In [13]:
num_classes = 15

import torchvision
def get_model(num_classes):
   # load an object detection model pre-trained on COCO
   model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# get the number of input features for the classifier
   in_features = model.roi_heads.box_predictor.cls_score.in_features
   print(in_features)
   # replace the pre-trained head with a new on
   model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
   
   return model

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# our dataset has two classes only - raccoon and not racoon
# get the model using our helper function
model = get_model(num_classes)
#move model to the right device
model.to(device)
# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler which decreases the learning rate by # 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

1024


In [14]:
# let's train it for 10 epochs
num_epochs = 1
for epoch in range(num_epochs):
   # train for one epoch, printing every 10 iterations
   metric = train_one_epoch(model, optimizer,data_loader_train , device, epoch, print_freq=10)
# update the learning rate
   lr_scheduler.step()
   # evaluate on the test dataset
   evaluate(model, data_loader_valid, device=device)

Epoch: [0]  [  0/352]  eta: 0:44:16  lr: 0.000019  loss: 2.3215 (2.3215)  loss_classifier: 2.1679 (2.1679)  loss_box_reg: 0.0195 (0.0195)  loss_objectness: 0.1258 (0.1258)  loss_rpn_box_reg: 0.0083 (0.0083)  time: 7.5478  data: 5.0672  max mem: 7159
Epoch: [0]  [ 10/352]  eta: 0:11:11  lr: 0.000162  loss: 2.1718 (2.0331)  loss_classifier: 1.9403 (1.7881)  loss_box_reg: 0.0146 (0.0127)  loss_objectness: 0.1825 (0.2199)  loss_rpn_box_reg: 0.0102 (0.0124)  time: 1.9638  data: 0.5453  max mem: 7425
Epoch: [0]  [ 20/352]  eta: 0:09:17  lr: 0.000304  loss: 0.8332 (1.2931)  loss_classifier: 0.7022 (1.0353)  loss_box_reg: 0.0123 (0.0129)  loss_objectness: 0.1825 (0.2331)  loss_rpn_box_reg: 0.0103 (0.0118)  time: 1.3869  data: 0.0876  max mem: 7425
Epoch: [0]  [ 30/352]  eta: 0:08:44  lr: 0.000446  loss: 0.2661 (0.9574)  loss_classifier: 0.0613 (0.7175)  loss_box_reg: 0.0120 (0.0131)  loss_objectness: 0.1794 (0.2155)  loss_rpn_box_reg: 0.0101 (0.0114)  time: 1.4452  data: 0.1301  max mem: 7425


In [59]:
metric

In [15]:
#save a model
PATH = "/kaggle/working/state_dict_model44.pt"
torch.save(model.state_dict(), PATH)

In [17]:
#load a trained model
PATH = "/kaggle/working/state_dict_model44.pt"
model = get_model(num_classes)
model.load_state_dict(torch.load(PATH))

1024


<All keys matched successfully>

In [28]:
# #test model on test data
# test_df = pd.read_csv('/kaggle/input/vinbigdata-original-image-dataset/vinbigdata/test.csv')
# test_df.head(), len(test_df)
# aa = test_df['image_id'] + '.jpg'
# aa = aa.to_list()

# path= "/kaggle/input/vinbigdata-original-image-dataset/vinbigdata"
# for i in range(2): #len(aa)
#     img_path = os.path.join(path, "test", aa[i])
#     img = Image.open(img_path).convert("RGB")
#     img = img.resize((512,512))
#     img = torch.from_numpy(np.array(img))
#     img = img.reshape(3, 512, 512)
#     print(img.shape, type(img))
# #     model.eval()
# #     with torch.no_grad():
# #         prediction = model([img.to(device)])


torch.Size([3, 512, 512]) <class 'torch.Tensor'>
torch.Size([3, 512, 512]) <class 'torch.Tensor'>


In [50]:
# pick one image from the test set
img, _ = dataset_valid[0]
# put the model in evaluation mode
model.eval()
with torch.no_grad():
    prediction = model([img])
prediction

[{'boxes': tensor([], size=(0, 4)),
  'labels': tensor([], dtype=torch.int64),
  'scores': tensor([])}]

In [46]:
type(img)

torch.Tensor